In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

import igraph
import pickle

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
data_file = 'data/cora_full.npz'
# data_file = 'data/reddit.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.25
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
topk=2
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

In [7]:
import warnings
warnings.filterwarnings("ignore")

# Load the data

In [8]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

print('shape attibute matrix: ', attr_matrix.n_rows)
print('Training: ', len(train_idx))
print('Validation: ', len(val_idx))
print('Testing: ', len(test_idx))

print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

70
shape attibute matrix:  18800
Training:  1400
Validation:  14000
Testing:  3400
train_idx:  [   12    16    18 ... 18790 18793 18798]
Runtime: 0.41s


# Preprocessing: Calculate PPR scores

In [9]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [10]:
from networkx import from_scipy_sparse_matrix, k_truss, core_number

g_networkX = from_scipy_sparse_matrix(adj_matrix)
# core_numbers = core_number(g_networkX)
# np.save('core-numbers-networkx', np.array(list(core_numbers.values())))


In [11]:
truss = k_truss(g_networkX, 3)
# print(truss.nodes[0:10])

In [12]:
#Compute core numbers
core_numbers = np.load('core-numbers-networkx.npy')

# graph = igraph.Graph.Adjacency((adj_matrix.todense()> 0).tolist())
# core_numbers = np.array(graph.coreness())




In [13]:
# with open('graph.pkl', 'wb') as outp:  # Overwrites any existing file.
#         pickle.dump(graph, outp, pickle.HIGHEST_PROTOCOL)

with open('graph.pkl', 'rb') as inp:
    graph = pickle.load(inp)

# shortest_path = graph.get_shortest_paths(0, to=[1,2,3])
# print(shortest_path)

In [14]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR

start = time.time()

topk_train, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, core_numbers, graph,
                                 normalization=ppr_normalization, S=1, gamma=1.0)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")


print('topk_train', topk_train.shape)
adj_xd = adj_matrix[train_idx]
print(adj_xd[0:0 + batch_size].shape)

Mean kn:  14.292142857142856
gamma:  1.0
Overall len y:  188.175 max:  491  min:  15
Runtime: 13.41s
topk_train (1400, 18800)
(512, 18800)


# Training: Set up model and train

In [15]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, weight_decay, dropout, adj_matrix, intermediate_layer=1000,
                    sparse_features=type(attr_matrix) is not np.ndarray)

# print(model.w0.shape)

# print(model.w_1.shape)

2022-07-11 11:21:14 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_12653/72873882.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-07-11 11:21:14 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_12653/72873882.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



In [16]:
sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val,
            labels=labels, adj_matrix=adj_matrix,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-07-11 11:21:15 (INFO): Epoch 6, step 20: train 3.96785
2022-07-11 11:21:16 (INFO): Epoch 13, step 40: train 3.33605
2022-07-11 11:21:16 (INFO): Epoch 19, step 60: train 2.87789
2022-07-11 11:21:17 (INFO): Epoch 26, step 80: train 2.57143
2022-07-11 11:21:17 (INFO): Epoch 33, step 100: train 1.82492
2022-07-11 11:21:18 (INFO): Epoch 39, step 120: train 1.56664
2022-07-11 11:21:18 (INFO): Epoch 46, step 140: train 1.52200
2022-07-11 11:21:19 (INFO): Epoch 53, step 160: train 1.14339
2022-07-11 11:21:20 (INFO): Epoch 59, step 180: train 1.03353
2022-07-11 11:21:20 (INFO): Epoch 66, step 200: train 1.10534
2022-07-11 11:21:21 (INFO): Epoch 73, step 220: train 0.88886
2022-07-11 11:21:22 (INFO): Epoch 79, step 240: train 0.85001
2022-07-11 11:21:22 (INFO): Epoch 86, step 260: train 0.92677
2022-07-11 11:21:23 (INFO): Epoch 93, step 280: train 0.78071
2022-07-11 11:21:24 (INFO): Epoch 99, step 300: train 0.75670
2022-07-11 11:21:24 (INFO): Epoch 106, step 320: train 0.80987
2022-07-11 1

Runtime: 17.88s


# Inference (val and test)

In [17]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 0.21s


# Collect and print results

In [18]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [19]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 99.1%, val: 60.8%, test: 61.0%
F1 score: Train: 0.985, val: 0.498, test: 0.498

Runtime: Preprocessing: 13.41s, training: 17.88s, inference: 0.21s -> total: 31.50s
Memory: Main: 0.57GB

